# Chapter 05 — Physics Libraries

- Engines (Box2D, Matter.js) manage collision detection, integration, and constraints.
- Use bodies (circles/polygons), fixtures (material properties), and joints (springs, rods).
- Wrap engines behind lightweight adapters to keep creative code expressive.
- Scale and units matter: map pixels to meters consistently.

Try: Replace the adapter with a real engine in a full environment; keep interfaces similar.

In [ ]:
# Minimal stand-in showing how an adapter might look without real engine dependency.from dataclasses import dataclassclass Vector:    def __init__(self, x=0.0, y=0.0):        self.x, self.y = float(x), float(y)    def __add__(self, other):        return Vector(self.x + other.x, self.y + other.y)    def __mul__(self, k):        return Vector(self.x * k, self.y * k)    __rmul__ = __mul__    def __repr__(self):        return f"Vector({self.x:.2f}, {self.y:.2f})"@dataclassclass BodyConfig:    position: tuple    shape: str = "circle"    radius: float = 1.0    density: float = 1.0    friction: float = 0.3class FakePhysicsWorld:    def __init__(self, gravity=(0, -9.8)):        self.gravity = gravity        self.bodies = []    def create_body(self, cfg: BodyConfig):        body = {"cfg": cfg, "vel": Vector(0, 0)}        self.bodies.append(body)        return body    def step(self, dt=1/60):        for body in self.bodies:            gx, gy = self.gravity            body["vel"] += Vector(gx * dt, gy * dt)world = FakePhysicsWorld()ball = world.create_body(BodyConfig(position=(0, 5), radius=0.5))for i in range(3):    world.step()    print(f"step {i}: vel={ball['vel']}")